## Imports

- This Notebook is being used to import and clean and data from the NBA API and then send it to Supabase

In [36]:
import pandas as pd
import requests
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import teams
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import boxscoretraditionalv2
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
from playwright._impl._errors import TimeoutError
import time
import os

## Headers For API Calls

In [2]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

### Functions Used

In [37]:
def find_backend_dir(start_path=None):
    """
    Walk up directories from start_path (or cwd) until a folder named 'backend' is found.
    Returns the absolute path to the 'backend' folder.
    Raises FileNotFoundError if not found.
    """
    if start_path is None:
        start_path = os.getcwd()
    curr_path = os.path.abspath(start_path)
    while True:
        # Check if 'backend' exists in this directory
        candidate = os.path.join(curr_path, "backend")
        if os.path.isdir(candidate):
            return candidate
        # If at filesystem root, stop
        parent = os.path.dirname(curr_path)
        if curr_path == parent:
            break
        curr_path = parent
    raise FileNotFoundError("No 'backend' directory found upward from {}".format(start_path))

In [3]:
# create function that gets player info data
def get_player_data(nba_player_id):
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=nba_player_id, headers=headers,timeout=200)
    df = player_info.common_player_info.get_data_frame()
    return df

In [4]:
# create function that gets player career stats
def get_player_career_stats(nba_player_id):
    career_stats = playercareerstats.PlayerCareerStats(player_id=nba_player_id, headers=headers, timeout=200)
    df = career_stats.get_data_frames()[0]
    return df

In [5]:
# create function that gets box score data based on game_ids
def get_box_score_data(game_id):
    player_stat_data = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id, headers=headers, timeout=200)
    df = player_stat_data.player_stats.get_data_frame()[0]
    return df

In [6]:
# Function to scrape table data from a page
async def scrape_table_data(page):
    html = await page.inner_html("body")
    soup = BeautifulSoup(html, "html.parser")
    rows = soup.select("tbody.Crom_body__UYOcU tr")
    data = [[td.get_text() for td in row.find_all("td")] for row in rows]
    return data

In [45]:
async def main(starting_season, forecast_season):
    season = starting_season
    boxscores_per_season = {}

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        while season != forecast_season:
            print(f"Processing season: {season}")
            url = "https://www.nba.com/stats/players/boxscores?SeasonType=Regular+Season"
            await page.goto(url, timeout=120000)
            await page.wait_for_selector("select.DropDown_select__4pIg9", timeout=10000)

            # --- PRINT AVAILABLE SEASONS ---
            options = await page.query_selector_all("select.DropDown_select__4pIg9 option")
            option_values = [await opt.inner_text() for opt in options]
            print("Available seasons in dropdown:", option_values)
            if season not in option_values:
                raise ValueError(f"Season '{season}' not in dropdown options! Check your season formatting.")

            # --- REMEMBER THE TABLE'S FIRST DATE (if present) ---
            first_row_date = await page.evaluate("""
                () => {
                    const row = document.querySelector("tbody.Crom_body__UYOcU tr");
                    if (row) {
                        const dateCell = row.querySelectorAll("td")[3];
                        return dateCell ? dateCell.innerText : null;
                    }
                    return null;
                }
            """)

            # --- SELECT THE SEASON + FIRE CHANGE EVENT ---
            await page.select_option("select.DropDown_select__4pIg9", label=season)
            await page.evaluate("""
                (season) => {
                    const sel = document.querySelector('select.DropDown_select__4pIg9');
                    sel.value = season;
                    sel.dispatchEvent(new Event('change', { bubbles: true }));
                }
            """, season)

            # --- WAIT FOR TABLE TO UPDATE (first row's date changes) ---
            start = time.time()
            while True:
                new_row_date = await page.evaluate("""
                    () => {
                        const row = document.querySelector("tbody.Crom_body__UYOcU tr");
                        if (row) {
                            const dateCell = row.querySelectorAll("td")[3];
                            return dateCell ? dateCell.innerText : null;
                        }
                        return null;
                    }
                """)
                if new_row_date != first_row_date or (time.time() - start) > 10:
                    break
                await asyncio.sleep(0.5)

            await asyncio.sleep(1.5)  # Give extra time for React to finish updating

            data = []
            page_number = 1

            while True:
                try:
                    # Scrape table data from the current page
                    data += await scrape_table_data(page)
                    print(f"Scraped data from page {page_number}.")

                    # Find next button
                    next_button = await page.query_selector("button[data-track='click'][data-type='controls'][data-pos='next']")
                    if not next_button:
                        print("No next button found. Exiting loop.")
                        break

                    await next_button.click(timeout=5000)
                    await page.wait_for_selector("tbody.Crom_body__UYOcU tr")
                    page_number += 1

                except Exception as e:
                    print(f"An error occurred: {e}")
                    break

            columns = [
                "Player", "Team", "Opponent", "Date", "Result", "Minutes", "Points", 
                "FGM", "FGA", "FG%", "3PM", "3PA", "3P%", "FTM", "FTA", "FT%", 
                "OREB", "DREB", "REB", "AST", "TO", "STL", "BLK", "PF", "+/-", "SPI"
            ]
            boxscores_df = pd.DataFrame(data, columns=columns)
            boxscores_per_season[season] = boxscores_df
            print(f"Scraping completed for season: {season}. Total games scraped: {len(boxscores_df)}")

            # Increment season (e.g., "2003-04" -> "2004-05")
            first_year = int(season[:4]) + 1
            second_year = int(season[5:]) + 1
            season = f"{first_year}-{second_year:02d}"

        await browser.close()
    return boxscores_per_season


## Finding Path

In [ ]:
backend_dir = find_backend_dir()       
csv_dir = os.path.join(backend_dir, "CSVs")

Backend directory: /Users/jeevanparmar/Uni/MSE 436/Project-Mono-Repo/backend
CSVs directory: /Users/jeevanparmar/Uni/MSE 436/Project-Mono-Repo/backend/CSVs


## Getting All Active Players

In [8]:
#  get active players function
active_players = players.get_active_players()

In [9]:
player_data = []
for nba_player in active_players:
    player_info = get_player_data(nba_player['id'])
    player_data.append(player_info)
    #time.sleep(3)

active_players_df = pd.concat(player_data, ignore_index=True)

In [10]:
active_players_df.sample(5)

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
335,1630241,Sam,Merrill,Sam Merrill,"Merrill, Sam",S. Merrill,sam-merrill,1996-05-15T00:00:00,Utah State,USA,...,sam_merrill,2020,2025,Y,Y,Y,2020,2,60,N
81,1627936,Alex,Caruso,Alex Caruso,"Caruso, Alex",A. Caruso,alex-caruso,1994-02-28T00:00:00,Texas A&M,USA,...,alex_caruso,2017,2025,Y,Y,Y,Undrafted,Undrafted,Undrafted,N
398,1629673,Jordan,Poole,Jordan Poole,"Poole, Jordan",J. Poole,jordan-poole,1999-06-19T00:00:00,Michigan,USA,...,jordan_poole,2019,2025,Y,Y,Y,2019,1,28,N
115,203076,Anthony,Davis,Anthony Davis,"Davis, Anthony",A. Davis,anthony-davis,1993-03-11T00:00:00,Kentucky,USA,...,anthony_davis,2012,2025,N,Y,Y,2012,1,1,Y
399,1641854,Craig,Porter Jr.,Craig Porter Jr.,"Porter Jr., Craig",C. Porter Jr.,craig-porter-jr,2000-02-26T00:00:00,Wichita State,USA,...,craig_porter,2023,2025,Y,Y,Y,Undrafted,Undrafted,Undrafted,N


In [11]:
# Adding Age For Each Player
active_players_df['CURRENT_AGE'] = pd.to_datetime(active_players_df['BIRTHDATE']).apply(lambda x: (pd.Timestamp.now() - x).days // 365)

In [12]:
active_players_df.sample(5)

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG,CURRENT_AGE
283,1630249,Vít,Krejčí,Vít Krejčí,"Krejčí, Vít",V. Krejčí,vít-krejčí,2000-06-19T00:00:00,Zaragoza,Czech Republic,...,2021,2025,Y,Y,Y,2020,2,37,N,25
458,1627783,Pascal,Siakam,Pascal Siakam,"Siakam, Pascal",P. Siakam,pascal-siakam,1994-04-02T00:00:00,New Mexico State,Cameroon,...,2016,2025,Y,Y,Y,2016,1,27,N,31
108,203109,Jae,Crowder,Jae Crowder,"Crowder, Jae",J. Crowder,jae-crowder,1990-07-06T00:00:00,Marquette,USA,...,2012,2025,Y,Y,Y,2012,2,34,N,34
490,1630679,Ethan,Thompson,Ethan Thompson,"Thompson, Ethan",E. Thompson,ethan-thompson,1999-05-04T00:00:00,Oregon State,Puerto Rico,...,2024,2025,Y,N,Y,Undrafted,Undrafted,Undrafted,N,26
52,1631128,Christian,Braun,Christian Braun,"Braun, Christian",C. Braun,christian-braun,2001-04-17T00:00:00,Kansas,USA,...,2022,2025,N,Y,Y,2022,1,21,N,24


In [13]:
active_players_df = active_players_df[['PERSON_ID', 'DISPLAY_FIRST_LAST', 'SCHOOL', 'TEAM_ID', 'TEAM_ABBREVIATION', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER','FROM_YEAR', 'TO_YEAR', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTHDATE', 'CURRENT_AGE']]

In [14]:
active_players_df.sample(5)

,PERSON_ID,DISPLAY_FIRST_LAST,SCHOOL,TEAM_ID,TEAM_ABBREVIATION,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,FROM_YEAR,TO_YEAR,POSITION,HEIGHT,WEIGHT,BIRTHDATE,CURRENT_AGE
182,1630224,Jalen Green,NBA G League Ignite,1610612745,HOU,2021,1,2,2021,2025,Guard,6-4,186,2002-02-09T00:00:00,23
285,1628398,Kyle Kuzma,Utah,1610612749,MIL,2017,1,27,2017,2025,Forward,6-9,221,1995-07-24T00:00:00,29
93,203903,Jordan Clarkson,Missouri,1610612762,UTA,2014,2,46,2014,2025,Guard,6-3,194,1992-06-07T00:00:00,33
161,1642277,Johnny Furphy,Kansas,1610612754,IND,2024,2,35,2024,2025,Guard,6-9,200,2004-12-08T00:00:00,20
473,1630531,Jaden Springer,Tennessee,1610612762,UTA,2021,1,28,2021,2025,Guard,6-4,202,2002-09-25T00:00:00,22


In [ ]:
# sending to csv
active_players_df.to_csv(os.path.join(csv_dir, 'active_players.csv'), index=False)

## Getting All Teams

In [18]:
# Get all teams.
nba_teams = teams.get_teams()
nba_teams_df = pd.DataFrame(nba_teams)

In [19]:
nba_teams_df

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [20]:
nba_teams_df = nba_teams_df[['id', 'full_name', 'abbreviation', 'nickname', 'city', 'state']]

In [ ]:
# Sending to csv
nba_teams_df.to_csv(os.path.join(csv_dir, 'nba_teams.csv'), index=False)

## Getting All Player Career Stats 

In [22]:
career_stats = []
for nba_player in active_players:
    player_stats = get_player_career_stats(nba_player['id'])
    career_stats.append(player_stats)
    #time.sleep(3)  

career_stats_df = pd.concat(career_stats, ignore_index=True)

/var/folders/8z/yxhm1c6x4bvg_s6d6w0cl6_80000gn/T/ipykernel_15217/3729197503.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  career_stats_df = pd.concat(career_stats, ignore_index=True)


In [23]:
career_stats_df.sample(5)

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
3429,1631102,2023-24,00,1610612749,MIL,22.0,11,0,56.0,6,...,0.000,1,4,5,5,3,0,2,1,14
1999,1641726,2024-25,00,1610612742,DAL,21.0,36,29,833.0,134,...,0.630,104,166,270,85,20,56,41,100,314
471,202692,2023-24,00,1610612765,DET,32.0,43,0,901.0,160,...,0.903,16,94,110,68,21,11,32,64,540
1917,1629111,2023-24,00,1610612745,HOU,28.0,56,3,764.0,106,...,0.800,88,84,172,65,20,33,31,74,274
2787,1629674,2021-22,00,1610612758,SAC,22.0,15,0,120.0,17,...,0.647,14,17,31,6,1,8,6,18,45


In [39]:
# sending to csv
career_stats_df.to_csv(os.path.join(csv_dir, 'career_stats.csv'), index=False)

## Getting Box Score Data for Every Game For Every Active Player Starting At The Earliest Season Recorded By Any Active Player

In [40]:
# Getting the earliest season for any active player
earliest_year = active_players_df['FROM_YEAR'].min()
earliest_season_years = f"{earliest_year}-{int(earliest_year + 1)}"
starting_season = earliest_season_years[:5]+earliest_season_years[7:]

In [41]:
starting_season

'2003-04'

In [42]:
# The upcoming season that's going to be forecasted
current_season = pd.Timestamp.now().year
current_season_years = f"{current_season}-{int(current_season + 1)}"
forecast_season = current_season_years[:5]+current_season_years[7:]

In [43]:
forecast_season

'2025-26'

In [46]:
boxscores_per_season = await main(starting_season, forecast_season)

Processing season: 2003-04
Available seasons in dropdown: ['2024-25', '2023-24', '2022-23', '2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12', '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06', '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00', '1998-99', '1997-98', '1996-97', '1995-96', '1994-95', '1993-94', '1992-93', '1991-92', '1990-91', '1989-90', '1988-89', '1987-88', '1986-87', '1985-86', '1984-85', '1983-84', '1982-83', '1981-82', '1980-81', '1979-80', '1978-79', '1977-78', '1976-77', '1975-76', '1974-75', '1973-74', '1972-73', '1971-72', '1970-71', '1969-70', '1968-69', '1967-68', '1966-67', '1965-66', '1964-65', '1963-64', '1962-63', '1961-62', '1960-61', '1959-60', '1958-59', '1957-58', '1956-57', '1955-56', '1954-55', '1953-54', '1952-53', '1951-52', '1950-51', '1949-50', '1948-49', '1947-48', '1946-47', 'Preseason', 'Regular Season', 'Playoffs', 'All-Star', 'Play In', 'NBA C

In [47]:
# Send each season's box scores to a CSV
for season, df in boxscores_per_season.items():
    season_clean = season.replace("-", "_")
    df.to_csv(os.path.join(csv_dir, f"boxscores_{season_clean}.csv"), index=False)
    print(f"Box scores for {season} saved to CSV.")

Box scores for 2003-04 saved to CSV.
Box scores for 2004-05 saved to CSV.
Box scores for 2005-06 saved to CSV.
Box scores for 2006-07 saved to CSV.
Box scores for 2007-08 saved to CSV.
Box scores for 2008-09 saved to CSV.
Box scores for 2009-10 saved to CSV.
Box scores for 2010-11 saved to CSV.
Box scores for 2011-12 saved to CSV.
Box scores for 2012-13 saved to CSV.
Box scores for 2013-14 saved to CSV.
Box scores for 2014-15 saved to CSV.
Box scores for 2015-16 saved to CSV.
Box scores for 2016-17 saved to CSV.
Box scores for 2017-18 saved to CSV.
Box scores for 2018-19 saved to CSV.
Box scores for 2019-20 saved to CSV.
Box scores for 2020-21 saved to CSV.
Box scores for 2021-22 saved to CSV.
Box scores for 2022-23 saved to CSV.
Box scores for 2023-24 saved to CSV.
Box scores for 2024-25 saved to CSV.


In [ ]:
# season = starting_season
# boxscores_per_season = {}
# while season != forecast_season:
#     print(f"Processing season: {season}")
#     gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season, 
#                                               league_id_nullable='00', 
#                                               season_type_nullable='Regular Season', headers=headers, timeout=200)
#     games = gamefinder.get_data_frames()[0]
#     # get a list of the distinct game_ids
#     boxscores = []
#     game_ids = games['GAME_ID'].unique().tolist()
#     for game_id in game_ids:
#         player_stat_data = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id="0020301183", headers=headers, timeout=1000)
#         boxscore_data = player_stat_data.get_data_frames()[0]
#         boxscores.append(boxscore_data)
#     boxscores_df = pd.concat(boxscores, ignore_index=True)
#     boxscores_per_season[season] = boxscores_df
#     season = str(int(season[:4]) + 1) + '-' + str(int(season[5:]) + 1)